# Parse Document with `ai_parse_document`

* To learn more, visit the official `ai_parse_document` [documentation page](https://docs.databricks.com/aws/en/sql/language-manual/functions/ai_parse_document).
* This notebook shows how to use `ai_parse_document` on a sample PDF file.
* This notebook should be run natively on Databricks, using `Serverless` compute option, with at least version 3 env. 
* For this example notebook, I'm using [Nvidia's company overview presentation from August 2025](https://s201.q4cdn.com/141608511/files/doc_presentations/2025/08/Q226-NVDA-Company-Overview-Final.pdf).

In [0]:
catalog = "users"  # TODO
schema = "david_huang"  # TODO
volume = "ai_parse_doc_examples"  # TODO

In [0]:
display(dbutils.fs.ls(f"/Volumes/{catalog}/{schema}/{volume}/input/"))

In [0]:
spark.sql(
    f"""
    create or replace table {catalog}.{schema}.aipd_output_bronze
    SELECT
        path,
        ai_parse_document(
            content,
            map(
                'version', '2.0',
                'imageOutputPath', '/Volumes/{catalog}/{schema}/{volume}/output/',
                'descriptionElementTypes', '*'
            )
        ) as parsed_doc
    FROM READ_FILES('/Volumes/{catalog}/{schema}/{volume}/input/', format => 'binaryFile')
    ;
    """
)

In [0]:
display(
    spark.sql(
        f"""
    SELECT
        path,
        parsed_doc:document:pages,
        parsed_doc:document:elements,
        parsed_doc:error_status,
        parsed_doc:metadata
    FROM {catalog}.{schema}.aipd_output_bronze
    ;
    """
    )
)